# Going Deeper 03 AI Lyrics Generator
###### 온라인 코어 2기 박수경  

이번 과제에서는 가사 데이터셋을 학습해 새로운 가사를 작성해 주는 인공지능을 구성해 보기로 한다. 규칙이 먼저 생기고 그에 맞는 문법에 따르는 기계어와는 다르게 자연적으로 존재하는 사람의 언어를 자연어라고 한다.  자연어는 규칙에 따라 생겨나는 것이 아니며, 따라서 통계적으로 접근하는 것이 가장 좋은 방법으로 알려져 있다.  

이번 과제에 사용하는 라이브러리들을 먼저 import 한다.

In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf
import numpy as np
import os
import re #정규표현식
from sklearn.model_selection import train_test_split # 데이터셋 분리

print(tensorflow.__version__)

2.6.0


## Step 1. 데이터 준비 및 읽어오기

glob 모듈을 사용해서 파일을 읽어온다. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장한다.

In [ ]:

txt_file_path = 'data/lyrics/*'

txt_list = glob.glob(txt_file_path)
raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too ", "But it's just like my girl "]


## Step 2. 데이터 정제

가사 데이터를 정제하는 함수 preprocess_sentence()를 정의하고 각 문장에 대해 적용한다.

지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거한다. 일반적으로 가사의 한 문장은 그렇게 길지 않기 때문에 적당한 길이의 문장을 학습하게 하는 것이다. 과제에서 제시한 max 토큰의 개수는 15개로, maxlen =15 로 설정한다.

정규표현식에서 https://stackoverflow.com/questions/13566052/using-r-with-variables-in-re-sub

한 문장에서 띄어쓰기를 기준으로 잘라 토큰화를 한다고 했을 때, 다음과 같은 처리를 할 필요가 있다.  
> 1. 모든 글자들을 소문자로 바꾸고, 양쪽 공백을 지우기
> 2. 특수문자 양쪽에 공백을 넣기
> 3. 여러개의 공백은 하나의 공백으로 바꾸기
> 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
> 5. 양쪽 공백 지우기
> 6. 문장 시작에는 <start>, 끝에는 <end>를 넣어 표시하기

In [ ]:
# sentence에서 원하는 것만 남기기

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 다 소문자로 만들기,strip :공백 지우기,
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자의 양쪽에 공백을 넣기 (\1 : 첫번째로 매치된 것에 대해) 
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다 (^: 여)
    sentence = sentence.strip() # 공백을 지운다.
    sentence = '<start> ' + sentence + ' <end>' # 시작과 끝에 start,end 붙인다
    return sentence

In [14]:
# raw_corpus 안에 어떤 결과가 담겼는지 확인한다.

print(raw_corpus[:9])

['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too "]


 raw_corpus 에서 실제로 학습에 쓰일 문장들(elements)만 선택해서 corpus 리스트에 담는다. 이 때 가사가 아닌 line들은 corpus에 담지 않기 위해 여러 조건문을 사용하여 corpus에 append 한다. 

 일단 앞서 확인한 [Spoken Intro:] 같은 경우에도 가사가 아니다. 대체로 chorus: , verse: 같이 곡의 구성상의 역할을 알려주는 line은 가사가 아니므로 제외하고 싶었다.
 이런 line들의 경우 끝이 :], :), :, ], ) 등으로 끝나는데 )로 끝나는 경우는 가사에 속할 수도 있으므로 :], :), :, ] 로 끝나는 것을 제외하기로 결정했다. 

In [6]:

corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    #제외할 문장의 특성 -> continue: 건너뛰기

    if len(sentence) == 0: 
        continue
    if sentence[-1] == ":": 
        continue
    if len(sentence) == 0: 
        continue
    if sentence[-1] == "]": 
        continue

    if len(sentence) == 1: 
        continue

    if sentence[-2:] == ":]": 
        continue
    if sentence[-2] == ":)": 
        continue
    
    
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 몇 개 확인한다.
corpus[:10]

['<start> come on , come on <end>',
 '<start> you think you drive me crazy <end>',
 '<start> come on , come on <end>',
 '<start> you and whose army ? <end>',
 '<start> you and your cronies <end>',
 '<start> come on , come on <end>',
 '<start> holy roman empire <end>',
 '<start> come on if you think <end>',
 '<start> come on if you think <end>',
 '<start> you can take us on <end>']

### 토큰화 (Tokenize)  
텐서플로우의 Tokenizer와 pad_sequences를 사용한다.
가사에 사용되는 단어장의 크기는 20000으로 설정했다. 시간은 더 걸릴 수 있지만 풍부한 단어가 확보되어 더 괜찮은 작사봇이 될 수 있겠다고 생각했기 때문이다. 단어장에 포함되지 못한 단어는 unknown으로 처리한다.  
pad_sequence 의 패딩 처리방법은 default 가 'pre' 이다. 따라서 뒤에 패딩을 추가하기 위해 'post'를 옵션으로 준다.


In [7]:

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, \
        padding='post',maxlen=15)  # 길이가 짧은 문장은 뒤에서부터 패딩(post), 토큰의 최대길이: 15  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

NameError: name 'tf' is not defined

tokenizer.index_word 로 토큰들의 인덱스와 해당 단어를 딕셔너리로 확인할 수 있다. 스무개 정도만 확인해 보도록 한다.

In [ ]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break

정제된 문장을 토큰화까지 마치고 데이터를 텐서로 만들었다. 이제는 사이킷런의 train_test_split()으로 데이터를 분리하려고 한다. train_test_split()을 사용하려면 X, y(target)으로 데이터가 분리되어 있어야 하므로 tensor를 src_input과 tgt_input으로 분리한다.

In [ ]:

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]  

In [ ]:
print(src_input[0])
print(tgt_input[0])

## Step 3. 평가 데이터셋 분리

In [ ]:


enc_train, enc_val, dec_train, dec_val = \
    train_test_split(src_input, tgt_input, \
        test_size=0.2, random_state=1004, shuffle=True)


In [ ]:
#데이터가 잘 분리되었는지 확인
enc_train.shape
enc_val.shape 
dec_train.shape 
dec_val.shape

buffer_size, batch_size, epoch, iteration을 다시 공부하는 계기가 되었다.
(https://losskatsu.github.io/machine-learning/epoch-batch/)

- Batch Size : 1회 역전파에서 gradient를 모을 데이터의 개수  
- Buffer Size : 전체 데이터의 개수  

- VOCAB_SIZE : 단어들의 개수와 padding '0'까지 합쳐서 +1

In [ ]:
BUFFER_SIZE = len(src_input) # 전체 문장의 개수
BATCH_SIZE = 256 
steps_per_epoch = len(src_input) // BATCH_SIZE 

VOCAB_SIZE = tokenizer.num_words + 1   # padding '0' 까지 합쳐서 +1

In [ ]:

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
print(dataset) # (166,678,, 19) -> (19,)

dataset = dataset.shuffle(BUFFER_SIZE)
print(dataset)

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

## Step 4. 인공지능 만들기

모델을 구성한다. 자연어처리에 적절한 LSTM 을 사용하도록 한다. LSTM은 RNN에서 발전된 그것의 한 종류로 'cell state', 'gate'의 개념을 다시 정리할 수 있었다. 

#hidden_size1, hidden_size2


In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size): # hidden_size
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
       
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)

        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 512 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.

#hidden_size2 = 

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

In [ ]:
# 모델의 구조를 확인합니다.
model.summary()

옵티마이저로 adam 을 사용한다. 
loss 로는 SparseCategoricalCrossentropy, from_logits, reduction

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 학습 데이터의 label이 정수일 경우 사용하는 손실함수
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환을 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

## Step 5. 가사 생성

In [ ]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he") # 시작문장으로 he를 넣어 문장생성 함수 실행

## Discussion

## References


- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer (토크나이즈)  
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences (문장에 패딩주기)

- https://codetorial.net/tensorflow/natural_language_processing_in_tensorflow_01.html (tokenizer의 여러가지 사용법에 대해 찾아보고 토큰으로 인코딩하는 방법에 조금 더 익숙해질 수 있었다.)

- https://stats.stackexchange.com/questions/265400/deep-learning-how-does-beta-1-and-beta-2-in-the-adam-optimizer-affect-its-lear (아담의 베타값 설정)